In [2]:
! explorer .

## 簡単な例の実装
http://testpy.hatenablog.com/entry/2017/02/02/000000

<span class="burk">ポイントはセッションが開いている時に保存することである！！！</span>  
以下でエラーなしであることを確認


            UT.save_state( './test_model/test_model')
            UT.load_state( './test_model/test_model')

## 呼び出しテスト

In [1]:
import pandas as pd
import gym_trading
import gym
import sys
import itertools
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

import baselines.common.tf_util as U
import baselines.deepq.utils as UT


from baselines import deepq
from baselines.deepq.replay_buffer import ReplayBuffer
from baselines.common.schedules import LinearSchedule

csv = "data/EURUSD60.csv"

env = gym.make('trading-v0')
env.initialise_simulator(csv, trade_period=50, train_split=0.7)#初期はtrade_period＝50

def model(inpt, num_actions, scope, reuse=False):
    """This model takes as input an observation and returns values of all actions."""
    with tf.variable_scope(scope, reuse=reuse):
        out = inpt
        out = layers.fully_connected(out, num_outputs=128, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=64, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=32, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)
        return out
    
def run_test(env, act, episodes=1, final_test=False):
    obs = env.reset(train=False)
    start = env.sim.train_end_index + 1
    end = env.sim.count - 2

    for episode in range(episodes):
        done = False
        while done is False:
            action = act(obs[None])
            obs, reward, done, info = env.step(action)

        if not final_test:
            journal = pd.DataFrame(env.portfolio.journal)
            profit = journal["Profit"].sum()
            return env.portfolio.average_profit_per_trade, profit
        else:
            print("Training period  %s - %s" % (env.sim.date_time[start], env.sim.date_time[end]))
            print("Average Reward is %s" % (env.portfolio.average_profit_per_trade))

    if final_test:
        env.generate_summary_stats()
    
with U.make_session(8):
    
    act, train, update_target, debug = deepq.build_train(
        make_obs_ph=lambda name: UT.BatchInput(env.observation_space.shape, name=name),
        q_func=model,
        num_actions=env.action_space.n,
        optimizer=tf.train.AdamOptimizer(learning_rate=5e-4),
    )
    UT.load_state( './test_model/test_model')
#     replay_buffer = ReplayBuffer(50000)
#     # Create the schedule for exploration starting from 1 (every action is random) down to
#     # 0.02 (98% of actions are selected according to values predicted by the model).
#     exploration = LinearSchedule(schedule_timesteps=10000, initial_p=1.0, final_p=0.02)
#     # Initialize the parameters and copy them to the target network.
#     U.initialize()
#     update_target()

#     episode_rewards = [0.0]
#     obs = env.reset()
#     l_mean_episode_reward = []

    # Show off the result
#     env.generate_summary_stats()
    run_test(env, act, final_test=True)


C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


                       Return       ATR  Open Trade  Duration Trade
Date_Time                                                          
2013-12-02 02:00:00  0.421251  0.355142         0.0             0.0
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./test_model/test_model
Training period  2014-04-08 20:00:00 - 2014-05-30 22:00:00
Average Reward is -6.979999999999988
SUMMARY STATISTICS
Total Trades Taken:  19
Total Reward:  -250.9999999999986
Average Reward per Trade:  -13.2105263157894
Win Ratio: 36.84210526315789 %


[ { 'Entry Price': 1.3789799999999999,
    'Entry Time': Timestamp('2014-04-09 09:00:00'),
    'Exit Price': 1.38845,
    'Exit Time': Timestamp('2014-04-11 11:00:00'),
    'Profit': 91.7000000000009,
    'Trade Duration': 50,
    'Type': 'BUY',
    'reward': 105.60000000000089},
  { 'Entry Price': 1.38896,
    'Entry Time': Timestamp('2014-04-11 13:00:00'),
    'Exit Price': 1.3827399999999999,
    'Exit Time': Timestamp('2014-04-14 14:00:00'),
    'Profit': -65.20000000000114,
    'Trade Duration': 25,
    'Type': 'BUY',
    'reward': -61.80000000000109},
  { 'Entry Price': 1.38153,
    'Entry Time': Timestamp('2014-04-14 16:00:00'),
    'Exit Price': 1.37998,
    'Exit Time': Timestamp('2014-04-15 10:00:00'),
    'Profit': -18.499999999999403,
    'Trade Duration': 18,
    'Type': 'BUY',
    'reward': -6.099999999998104},
  { 'Entry Price': 1.3804,
    'Entry Time': Timestamp('2014-04-15 13:00:00'),
    'Exit Price': 1.3845299999999998,
    'Exit Time': Timestamp('2014-04-17 15:00:0

## 保存

In [1]:
import pandas as pd
import gym_trading
import gym
import sys
import itertools
import numpy as np
import tensorflow as tf
import tensorflow.contrib.layers as layers

import baselines.common.tf_util as U
import baselines.deepq.utils as UT

from baselines import deepq
from baselines.deepq.replay_buffer import ReplayBuffer
from baselines.common.schedules import LinearSchedule


def model(inpt, num_actions, scope, reuse=False):
    """This model takes as input an observation and returns values of all actions."""
    with tf.variable_scope(scope, reuse=reuse):
        out = inpt
        out = layers.fully_connected(out, num_outputs=128, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=64, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=32, activation_fn=tf.nn.tanh)
        out = layers.fully_connected(out, num_outputs=num_actions, activation_fn=None)
        return out


def run_test(env, act, episodes=1, final_test=False):
    obs = env.reset(train=False)
    start = env.sim.train_end_index + 1
    end = env.sim.count - 1

    for episode in range(episodes):
        done = False
        while done is False:
            action = act(obs[None])
            obs, reward, done, info = env.step(action)

        if not final_test:
            journal = pd.DataFrame(env.portfolio.journal)
            profit = journal["Profit"].sum()
            return env.portfolio.average_profit_per_trade, profit
        else:
            print("Training period  %s - %s" % (env.sim.date_time[start], env.sim.date_time[end]))
            print("Average Reward is %s" % (env.portfolio.average_profit_per_trade))

    if final_test:
        env.generate_summary_stats()


with U.make_session(8):
    # csv = "/home/adrian/Escritorio/polinex/EURUSD60.csv"
    csv = "data/EURUSD60.csv"

    env = gym.make('trading-v0')
    env.initialise_simulator(csv, trade_period=50, train_split=0.7)

    
    act, train, update_target, debug = deepq.build_train(
        make_obs_ph=lambda name: UT.BatchInput(env.observation_space.shape, name=name),
        q_func=model,
        num_actions=env.action_space.n,
        optimizer=tf.train.AdamOptimizer(learning_rate=5e-4),
    )
    UT.load_state( './test_model/test_model')
    
    replay_buffer = ReplayBuffer(50000)
    # Create the schedule for exploration starting from 1 (every action is random) down to
    # 0.02 (98% of actions are selected according to values predicted by the model).
    exploration = LinearSchedule(schedule_timesteps=10000, initial_p=1.0, final_p=0.02)
    # Initialize the parameters and copy them to the target network.
    U.initialize()
    update_target()

    episode_rewards = [0.0]
    obs = env.reset()
    l_mean_episode_reward = []
    for t in itertools.count():
        # Take action and update exploration to the newest value
        action = act(obs[None], update_eps=exploration.value(t))[0]

        new_obs, rew, done, _ = env.step(action)

        # Store transition in the replay buffer.
        replay_buffer.add(obs, action, rew, new_obs, float(done))

        obs = new_obs

        episode_rewards[-1] += rew

        is_solved = np.mean(episode_rewards[-101:-1]) > 5000 or t >= 100000
        is_solved = is_solved and len(env.portfolio.journal) != 0

        if done:

            journal = pd.DataFrame(env.portfolio.journal)
            profit = journal["Profit"].sum()

            try:
                print("-------------------------------------")
                print("steps                     | {:}".format(t))
                print("episodes                  | {}".format(len(episode_rewards)))
                print("% time spent exploring    | {}".format(int(100 * exploration.value(t))))

                print("--")
                l_mean_episode_reward.append(round(np.mean(episode_rewards[-101:-1]), 1))

                print("mean episode reward       | {:}".format(l_mean_episode_reward[-1]))
                print("Total operations          | {}".format(len(env.portfolio.journal)))
                print("Avg duration trades       | {}".format(round(journal["Trade Duration"].mean(), 2)))
                print("Total profit episode      | {}".format(round(profit), 1))
                print("Avg profit per trade      | {}".format(round(env.portfolio.average_profit_per_trade, 3)))

                print("--")

                reward_test, profit = run_test(env=env, act=act)
                print("Total profit test:        > {}".format(round(profit, 2)))
                print("Avg profit per trade test > {}".format(round(reward_test, 3)))
                print("-------------------------------------")
            except Exception as e:
                print("Exception: ", e)
                # Update target network periodically.

            obs = env.reset()
            episode_rewards.append(0)



        if is_solved:
            # Show off the result
            env.generate_summary_stats()
            run_test(env, act, final_test=True)
            UT.save_state( './test_model/test_model')
            
            break

        else:
            # Minimize the error in Bellman's equation on a batch sampled from replay buffer.
            if t > 500:
                obses_t, actions, rewards, obses_tp1, dones = replay_buffer.sample(32)
                train(obses_t, actions, rewards, obses_tp1, dones, np.ones_like(rewards))
            if t % 500 == 0:
                update_target()




C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\matplotlib\cbook\deprecation.py:106: MatplotlibDeprecationWarning: The finance module has been deprecated in mpl 2.0 and will be removed in mpl 2.2. Please use the module mpl_finance instead.
  warnings.warn(message, mplDeprecation, stacklevel=1)


                       Return       ATR  Open Trade  Duration Trade
Date_Time                                                          
2013-12-02 02:00:00  0.421251  0.355142         0.0             0.0
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
INFO:tensorflow:Restoring parameters from ./test_model/test_model


C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\numpy\core\fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
C:\Users\mineo\Anaconda2\envs\gym_test\lib\site-packages\numpy\core\_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


-------------------------------------
steps                     | 2138
episodes                  | 1
% time spent exploring    | 79
--
mean episode reward       | nan
Total operations          | 374
Avg duration trades       | 3.55
Total profit episode      | -1020.0
Avg profit per trade      | -2.584
--
Total profit test:        > -206.6
Avg profit per trade test > -0.933
-------------------------------------
-------------------------------------
steps                     | 4277
episodes                  | 2
% time spent exploring    | 58
--
mean episode reward       | -1272.7
Total operations          | 428
Avg duration trades       | 2.36
Total profit episode      | -1341.0
Avg profit per trade      | -3.138
--
Total profit test:        > -525.7
Avg profit per trade test > -2.953
-------------------------------------
-------------------------------------
steps                     | 6416
episodes                  | 3
% time spent exploring    | 37
--
mean episode reward       | -2100

Total profit test:        > 30.7
Avg profit per trade test > -0.537
-------------------------------------
-------------------------------------
steps                     | 44918
episodes                  | 21
% time spent exploring    | 2
--
mean episode reward       | -1255.2
Total operations          | 28
Avg duration trades       | 1.0
Total profit episode      | -88.0
Avg profit per trade      | -3.343
--
Total profit test:        > -11.1
Avg profit per trade test > -4.78
-------------------------------------
-------------------------------------
steps                     | 47057
episodes                  | 22
% time spent exploring    | 2
--
mean episode reward       | -1199.9
Total operations          | 22
Avg duration trades       | 1.0
Total profit episode      | -91.0
Avg profit per trade      | -5.495
--
Total profit test:        > -84.8
Avg profit per trade test > -3.833
-------------------------------------
-------------------------------------
steps                     | 4

Total profit test:        > -117.1
Avg profit per trade test > -1.794
-------------------------------------
-------------------------------------
steps                     | 87698
episodes                  | 41
% time spent exploring    | 2
--
mean episode reward       | -689.3
Total operations          | 31
Avg duration trades       | 1.0
Total profit episode      | -122.0
Avg profit per trade      | -5.71
--
Total profit test:        > -80.3
Avg profit per trade test > -0.72
-------------------------------------
-------------------------------------
steps                     | 89837
episodes                  | 42
% time spent exploring    | 2
--
mean episode reward       | -676.8
Total operations          | 51
Avg duration trades       | 1.33
Total profit episode      | -201.0
Avg profit per trade      | -3.045
--
Total profit test:        > -53.0
Avg profit per trade test > -5.943
-------------------------------------
-------------------------------------
steps                     |

[ { 'Entry Price': 1.35881,
    'Entry Time': Timestamp('2013-12-02 03:00:00'),
    'Exit Price': 1.3598700000000001,
    'Exit Time': Timestamp('2013-12-02 04:00:00'),
    'Profit': 7.600000000000609,
    'Trade Duration': 1,
    'Type': 'BUY',
    'reward': 13.199999999999548},
  { 'Entry Price': 1.3787,
    'Entry Time': Timestamp('2013-12-11 22:00:00'),
    'Exit Price': 1.37856,
    'Exit Time': Timestamp('2013-12-11 23:00:00'),
    'Profit': -4.40000000000029,
    'Trade Duration': 1,
    'Type': 'BUY',
    'reward': -5.40000000000018},
  { 'Entry Price': 1.3783299999999998,
    'Entry Time': Timestamp('2013-12-12 11:00:00'),
    'Exit Price': 1.3772,
    'Exit Time': Timestamp('2013-12-12 12:00:00'),
    'Profit': -14.299999999998533,
    'Trade Duration': 1,
    'Type': 'BUY',
    'reward': -19.699999999998383},
  { 'Entry Price': 1.3774600000000001,
    'Entry Time': Timestamp('2013-12-12 17:00:00'),
    'Exit Price': 1.37511,
    'Exit Time': Timestamp('2013-12-12 18:00:00'),

[ { 'Entry Price': 1.37943,
    'Entry Time': Timestamp('2014-04-09 15:00:00'),
    'Exit Price': 1.3812799999999998,
    'Exit Time': Timestamp('2014-04-09 16:00:00'),
    'Profit': 15.499999999999073,
    'Trade Duration': 1,
    'Type': 'BUY',
    'reward': 3.3000000000013046},
  { 'Entry Price': 1.3799700000000001,
    'Entry Time': Timestamp('2014-04-15 11:00:00'),
    'Exit Price': 1.38037,
    'Exit Time': Timestamp('2014-04-15 12:00:00'),
    'Profit': -6.9999999999995595,
    'Trade Duration': 1,
    'Type': 'SELL',
    'reward': 3.8000000000023597},
  { 'Entry Price': 1.38153,
    'Entry Time': Timestamp('2014-04-18 22:00:00'),
    'Exit Price': 1.38091,
    'Exit Time': Timestamp('2014-04-18 23:00:00'),
    'Profit': 3.199999999998429,
    'Trade Duration': 1,
    'Type': 'SELL',
    'reward': 1.3999999999999595},
  { 'Entry Price': 1.38334,
    'Entry Time': Timestamp('2014-04-25 08:00:00'),
    'Exit Price': 1.3831200000000001,
    'Exit Time': Timestamp('2014-04-25 09:00:

In [1]:
! explorer .